# Walker Lake

## The Disappearing Walker Lake
While the loss of the Aral Sea in Kazakhstan and Lake Urmia in Iran have received a lot of attention over the last few decades, this trend is a global phenomena.  Reciently a number of __[papers](https://earthobservatory.nasa.gov/IOTD/view.php?id=91921)__ have been published including one focusing on the __[Decline of the world's saline lakes](https://www.nature.com/articles/ngeo3052)__.  Many of these lakes have lost the majority of their volume over the last century, including Walker Lake (Nevada, USA) which has lost 90 percent of its volume over the last 100 years.

The following example is intended to replicate the typical processing required in change detection studies similar to the __[Decline of the world's saline lakes](https://www.nature.com/articles/ngeo3052)__.

In [ ]:
from pathlib import Path

import geoviews as gv
import holoviews as hv
import numpy as np
import rioxarray
import xarray as xr

import cartopy.crs as ccrs
from colorcet import coolwarm
from holoviews import opts
from holoviews.operation.datashader import rasterize
from IPython.display import display

hv.extension('bokeh')

In this example, we would like to use [Dask](https://dask.org/) to
demonstrate how image processing can be distributed across workers,
either running locally or across a cluster. In the next cell, we
instantiate a Dask distributed Client where we request eight,
single-threaded workers and declare a memory limit of 8GB per
worker. You can experiment with different memory limits (e.g 4GB) and
different numbers of workers but note that each worker should only use
one thread as [Datashader](https://datashader.org) manages its own
parallelization using [Numba](https://numba.pydata.org/):

In [ ]:
# arbitrarily choose a memory limit (8GB) to demonstrate the out of core
# processing infrastructure
from dask.distributed import Client
client = Client(memory_limit=8*1e9, n_workers=8, threads_per_worker=1)
# As Datashader uses parallel Numba for raster rendering, we need to use
# single threaded Dask workers on each CPU to avoid contention.
client

### Landsat Image Data

To replicate this study, we first have to obtain the data from primary sources.  The conventional way to obtain Landsat image data is to download it through USGS's 
__[EarthExplorer](https://earthexplorer.usgs.gov/)__ or NASA's __[Giovanni](https://giovanni.gsfc.nasa.gov/giovanni/)__, but to facilitate the example two images have been downloaded from EarthExployer and cached.  

The two images used by the original study are LT05_L1TP_042033_19881022_20161001_01_T1 and 
LC08_L1TP_042033_20171022_20171107_01_T1 from 1988/10/22 and 2017/10/22 respectively.  These images contain Landsat Surface Reflectance Level-2 Science Product images.

## Loading into xarray

In the next cells, we load the Landsat-5 and Landsat-8 files into xarray `DataArray` objects, reading them locally using `rioxarray`.

In [ ]:
def read_landsat_files(pattern):
    data_dir = Path('data')
    data = {
        int(file.stem[-1]): rioxarray.open_rasterio(file, chunks={"x": 1200, "y": 1200}, masked=True)
        for file in sorted(data_dir.glob(pattern))
    }
    dataset = xr.concat(data.values(), dim="band")
    dataset = dataset.assign_coords({"band": list(data)})
    return dataset

In [ ]:
landsat_5_img = read_landsat_files('LT05*')
landsat_5_img

In [ ]:
landsat_8_img = read_landsat_files('LC08*')
landsat_8_img

In [ ]:
assert landsat_5_img.rio.crs == landsat_8_img.rio.crs
print(landsat_5_img.rio.crs)

We create a cartopy coordinate reference system that we will be using later on in this notebook:

In [ ]:
crs = ccrs.epsg(landsat_5_img.rio.crs.to_epsg())

## Computing the NDVI (1988)

Now let us compute the [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) for the 1988 image.

In [ ]:
ndvi5 = (landsat_5_img[4] - landsat_5_img[3])/(landsat_5_img[4] + landsat_5_img[3])

In [ ]:
client.persist(ndvi5)

## Computing the NDVI (2017)

Now we can do this for the Landsat 8 files for the 2017 image:

In [ ]:
ndvi8 = (landsat_8_img[4] - landsat_8_img[3])/(landsat_8_img[4] + landsat_8_img[3])
# ndvi8 = ndvi8_array.to_dataset(name='ndvi')[['x','y', 'ndvi']]
# client.persist(ndvi8)

# Resampling to same size

The two images share the same coordinate system but do not have the exact same dimensions or coordinates. Previous versions of this notebook resampled the images to the same size, and optionally allowed to regrid them, all using  Datashader. In this version, we now interpolate the Landsat-8 image to fit onto the coordinates of the Landsat-5 one using `xarray`, approach that provides a similar result.

In [ ]:
ndvi8 = ndvi8.interp_like(ndvi5, method="nearest")
client.persist(ndvi8)

## Viewing change via dropdown

Using [Datashader](https://datashader.org/) together with [GeoViews](https://geoviews.org/), we can now easily build an interactive visualization where we select between the 1988 and 2017 images. The use of datashader allows these images to be dynamically updated according to zoom level (Note: it can take datashader a minute to 'warm up' before it becomes fully interactive). For more information on how the dropdown widget was created using `HoloMap`, please refer to the [HoloMap reference](https://holoviews.org/reference/containers/bokeh/HoloMap.html#bokeh-gallery-holomap).

In [ ]:
opts.defaults(
    opts.Curve(width=600, tools=['hover']),
    opts.Image(cmap='viridis', width=450, height=450, tools=['hover'], colorbar=True))

In [ ]:
hmap = hv.HoloMap({'1988':gv.Image(ndvi5, crs=crs, vdims=['ndvi'], rtol=10), 
                   '2017':gv.Image(ndvi8, crs=crs, vdims=['ndvi'], rtol=10)}, 
                  kdims=['Year']).redim(x='lon', y='lat') # Mapping 'x' and 'y' from rasterio to 'lon' and 'lat'

In [ ]:
%%time
display(rasterize(hmap))

## Computing statistics and projecting display

The rest of the notebook shows how statistical operations can reduce the dimensionality of the data that may be used to compute new features that may be used as part of an ML pipeline. 

### The mean and sum over the two time points

The next plot (may take a minute to compute) shows the mean of the two NDVI images next to the sum of them:

In [ ]:
mean_avg = hmap.collapse(dimensions=['Year'], function=np.mean)
mean_img = gv.Image(mean_avg.data, crs=crs, kdims=['lon', 'lat'], 
                    vdims=['ndvi']).relabel('Mean over Year')

summed = hmap.collapse(dimensions=['Year'], function=np.sum)
summed_image = gv.Image(summed.data, crs=crs, kdims=['lon', 'lat'], 
                        vdims=['ndvi']).relabel('Sum over Year')

In [ ]:
%%time
display(rasterize(mean_img) + rasterize(summed_image))

### Difference in NDVI between 1988 and 2017

The change in Walker Lake as viewed using the NDVI can be shown by subtracting the NDVI recorded in 1988 from the NDVI recorded in 2017:

In [ ]:
diff = hmap['1988'].data - hmap['2017'].data
difference = gv.Image(diff, crs=crs, kdims=['lon', 'lat'], vdims=['ndvi'])
difference = difference.relabel('Difference in NDVI').redim(ndvi='delta_ndvi')

In [ ]:
%%time
display(rasterize(difference).redim.range(delta_ndvi=(-1.0,1.0)).opts(cmap=coolwarm))

You can see a large change (positive delta) in the areas where there is water, indicating a reduction in the size of the lake over this time period.

## Slicing across `lon` and `lat`

As a final example, we can use the `sample` method to slice across the difference in NDVI along (roughly) the midpoint of the latitude and the midpoint of the longitude. To do this, we define the following helper function to convert latitude/longitude into the appropriate coordinate value used by the `DataSet`:

In [ ]:
def from_lon_lat(x,y):
    return crs.transform_point(x,y, ccrs.PlateCarree())

lon_y, lat_x = from_lon_lat(-118, 39)  # Longitude of -118 and Latitude of 39

In [ ]:
%%time
display((difference.sample(lat=lat_x) + difference.sample(lon=lon_y)).cols(1))